In [1]:
import math
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import IPython.display as disp
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from torchvision import transforms
from compressai.models import CompressionModel
from compressai.entropy_models import EntropyBottleneck
from compressai.models.utils import conv, deconv
from compressai.layers import GDN

In [ ]:
dataset_list = []
for batch in range(30):
    split = f"batch{batch}"
    dataset = load_dataset("danjacobellis/AVIRIS_256",split=split).with_format("torch");
    dataset_list.append(dataset)
dataset = concatenate_datasets(dataset_list)

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
class RateDistortionAutoEncoder(CompressionModel):
    def __init__(self, N=240):
        super().__init__()
        self.entropy_bottleneck = EntropyBottleneck(N)
        self.encode = nn.Sequential(
            conv(224, N),
            GDN(N),
            conv(N, N),
            GDN(N),
            conv(N, N),
        )

        self.decode = nn.Sequential(
            deconv(N, N),
            GDN(N, inverse=True),
            deconv(N, N),
            GDN(N, inverse=True),
            deconv(N, 224),
        )

    def forward(self, x):
        y = self.encode(x)
        y_hat, y_likelihoods = self.entropy_bottleneck(y)
        x_hat = self.decode(y_hat)
        return x_hat, y_likelihoods

In [ ]:
def live_plot(rate, distortion):
    disp.clear_output(wait=True)
    plt.figure(figsize=(10, 6))
    
    plt.subplot(2, 1, 1)
    plt.plot(np.log(rate))
    plt.title('Rate')
    plt.xlabel('Iteration')
    plt.ylabel('log(Bits per Sample)')
    
    plt.subplot(2, 1, 2)
    plt.plot(np.log(distortion))
    plt.title('Distortion')
    plt.xlabel('Iteration')
    plt.ylabel('log(Mean Square Error)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
net = RateDistortionAutoEncoder().to("cuda")
parameters = set(p for n, p in net.named_parameters() if not n.endswith(".quantiles"))
aux_parameters = set(p for n, p in net.named_parameters() if n.endswith(".quantiles"))
optimizer = optim.AdamW(parameters, lr=1e-3)
aux_optimizer = optim.AdamW(aux_parameters, lr=1e-3)
λ = 1e-5

In [ ]:
rate = np.array([])
distortion = np.array([])
for i_epoch in range(500):
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    for i,batch in enumerate(dataloader):
        optimizer.zero_grad()
        aux_optimizer.zero_grad()
    
        x = batch['hyperspectral'].to("cuda")
        x = x.to(torch.float)
        
        x_hat, y_likelihoods = net(x)
        
        # bitrate of the quantized latent
        N, _, H, W = x.size()
        num_pixels = N * H * W
        bpp_loss = torch.log(y_likelihoods).sum() / (-math.log(2) * num_pixels)
        rate = np.append(rate,bpp_loss.detach().cpu().numpy())
        
        # mean square error
        mse_loss = F.mse_loss(x, x_hat)
        distortion = np.append(distortion,mse_loss.detach().cpu().numpy())
        
        # final loss term
        loss = mse_loss + λ * bpp_loss
        
        loss.backward()
        optimizer.step()
        
        aux_loss = net.aux_loss()
        aux_loss.backward()
        aux_optimizer.step()

        torch.save({
                'epoch': i,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'aux_optimizer_state_dict': aux_optimizer.state_dict(),
                'loss': loss,
            }, f"rdae_AVIRIS_checkpoint.pth")
        live_plot(rate,distortion)
        np.save('rate',rate)
        np.save('distortion',distortion)